In [1]:
from pathlib import Path
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

In [2]:
CLASSES = ["cubes", "spheres"]
LABEL_MAP = {"cubes": 0, "spheres": 1}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

STORAGE_PATH = Path("/content/drive/MyDrive/Colab Notebooks/Applied Computer Vision/Applied-Computer-Vision-Projects/Multimodal_Learning_02/")

Mounted at /content/drive


In [4]:
DATA_SRC_PATH = STORAGE_PATH / "multimodal_training_workshop/data"
DATA_DEST_PATH = STORAGE_PATH / "data"
print(f"Data path: {DATA_SRC_PATH}")
print(f"Data path exists: {DATA_SRC_PATH.exists()}")

Data path: /content/drive/MyDrive/Colab Notebooks/Applied Computer Vision/Applied-Computer-Vision-Projects/Multimodal_Learning_02/multimodal_training_workshop/data
Data path exists: True


In [6]:
def get_torch_xyza(lidar_depth, azimuth, zenith):
    x = lidar_depth * torch.sin(-azimuth[:, None]) * torch.cos(-zenith[None, :])
    y = lidar_depth * torch.cos(-azimuth[:, None]) * torch.cos(-zenith[None, :])
    z = lidar_depth * torch.sin(-zenith[None, :])
    a = torch.where(lidar_depth < 50.0, torch.ones_like(lidar_depth), torch.zeros_like(lidar_depth))
    xyza = torch.stack((x, y, z, a))
    return xyza

In [9]:
from pathlib import Path
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

def precompute_assessment_xyza(
    src_root,
    dst_root,
    classes,
    label_map,
):
    """
    Convert assessment depth LiDAR (H,W) to XYZA (4,H,W) and save to dst_root.
    """
    src_root = Path(src_root)
    dst_root = Path(dst_root)
    dst_root.mkdir(parents=True, exist_ok=True)

    all_samples = []

    for class_name in classes:
        src_class_dir = src_root / class_name
        rgb_dir   = src_class_dir / "rgb"
        lidar_dir = src_class_dir / "lidar"

        # azimuth / zenith per class
        azimuth_path = src_class_dir / "azimuth.npy"
        zenith_path  = src_class_dir / "zenith.npy"
        if not azimuth_path.exists() or not zenith_path.exists():
            raise FileNotFoundError(f"Missing azimuth/zenith in {src_class_dir}")

        azimuth = torch.from_numpy(np.load(azimuth_path)).to(torch.float32)
        zenith  = torch.from_numpy(np.load(zenith_path)).to(torch.float32)

        # destination dirs
        dst_class_dir = dst_root / class_name
        dst_rgb_dir   = dst_class_dir / "rgb"
        dst_lidar_dir = dst_class_dir / "lidar_xyza"

        dst_rgb_dir.mkdir(parents=True, exist_ok=True)
        dst_lidar_dir.mkdir(parents=True, exist_ok=True)

        rgb_files   = sorted(rgb_dir.glob("*.png"))
        lidar_files = sorted(lidar_dir.glob("*.npy"))

        rgb_stems   = {f.stem for f in rgb_files}
        lidar_stems = {f.stem for f in lidar_files}
        matching    = sorted(rgb_stems & lidar_stems)

        print(f"{class_name}: {len(matching)} paired samples")

        for stem in tqdm(matching, desc=f"Converting {class_name}"):
            # --- RGB: copy to dst_root (or keep src, your choice) ---
            rgb_src = rgb_dir / f"{stem}.png"
            rgb_dst = dst_rgb_dir / f"{stem}.png"
            if not rgb_dst.exists():
                Image.open(rgb_src).save(rgb_dst)

            # --- depth → XYZA ---
            depth_src = lidar_dir / f"{stem}.npy"
            depth_np  = np.load(depth_src)              # (H, W)
            depth_t   = torch.from_numpy(depth_np).to(torch.float32)

            xyza = get_torch_xyza(depth_t, azimuth, zenith)  # (4, H, W)

            lidar_dst = dst_lidar_dir / f"{stem}.npy"
            np.save(lidar_dst, xyza.numpy())

            all_samples.append(
                {
                    "class": class_name,
                    "rgb": rgb_dst,
                    "lidar_xyza": lidar_dst,
                    "label": label_map[class_name],
                }
            )

    print(f"Total converted samples: {len(all_samples)}")
    return all_samples


In [10]:
SRC = DATA_SRC_PATH / "assessment"
DST = DATA_DEST_PATH

all_samples_xyza = precompute_assessment_xyza(src_root=SRC,
    dst_root=DST,
    classes=CLASSES,
    label_map=LABEL_MAP,)

cubes: 2501 paired samples


Converting cubes: 100%|██████████| 2501/2501 [20:09<00:00,  2.07it/s]


spheres: 9999 paired samples


Converting spheres: 100%|██████████| 9999/9999 [1:18:52<00:00,  2.11it/s]


Total converted samples: 12500


In [ ]:
print(len(all_samples_xyza))